In [1]:
%load_ext autoreload
%autoreload 1
%aimport sentiment_utils

[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import os
import requests

import pandas as pd
import sklearn
from tqdm import tqdm
import torch

print('python:'.ljust(16), sys.version.split('\n')[0])
print('Scikit-learn:'.ljust(16), sklearn.__version__)
print('PyTorch:'.ljust(16), torch.__version__)

python:          3.10.8 (main, Oct 13 2022, 09:48:40) [Clang 14.0.0 (clang-1400.0.29.102)]
Scikit-learn:    1.2.2
PyTorch:         1.13.1


## Device

In [3]:
# Get cpu or gpu device for training
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {DEVICE} device')

Using cpu device


## Hyperparameters & Constants

In [4]:
# Hyperparameters
# BATCH_SIZE = 64
# EPOCHS = 15  # select from: 2**n - 1 = [1, 3, 7, 15, ...]
# SCHEDULER_GAMMA = 0.7

# Constants
CLASSES = 3
DATASET_NAME = 'tweet_eval'
DATASET_PATH = f'./data/{DATASET_NAME}/'

# Actions
# DO_LR_RANGE_TEST=False

## Random state

In [5]:
RANDOM_STATE = 2147483647
# random.seed(RANDOM_STATE)
# np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed_all(RANDOM_STATE)

## Load data

In [6]:
sentiment_utils.download_dataset(DATASET_NAME, DATASET_PATH)
df_train = pd.read_parquet(DATASET_PATH + 'tweet_eval-train.parquet')
df_valid = pd.read_parquet(DATASET_PATH + 'tweet_eval-validation.parquet')
df_test = pd.read_parquet(DATASET_PATH + 'tweet_eval-test.parquet')

In [7]:
df_train.head()

,text,label
0,"""QT @user In the original draft of the 7th boo...",2
1,"""Ben Smith / Smith (concussion) remains out of...",1
2,Sorry bout the stream last night I crashed out...,1
3,Chase Headley's RBI double in the 8th inning o...,1
4,@user Alciato: Bee will invest 150 million in ...,2


## Tokenize

In [27]:
%%time
tokenizer = sentiment_utils.Tokenizer()

text = u"Is 9.5 or 525,600 my favorite number? This is it and I didn't do so as @dkjdkr. Yes takes it!"
print(tokenizer(text, return_str=True))
print()
text = u'The https://github.com/jonsafari/tok-tok/blob/master/tok-tok.pl is a website with/and/or slashes and sort of weird : things'
print(tokenizer(text, return_str=True))
print()
text = u'¡This, is a sentence with weird» symbols… appearing @ everywhere¿ ))'
print(tokenizer(text, return_str=True))
print()

9.5 525,600 favorit number ? ' @user ye take !

http websit with/and/or slash sort weird : thing

¡ , sentenc weird » symbol … appear @ everywher ¿ ) )

CPU times: user 632 µs, sys: 546 µs, total: 1.18 ms
Wall time: 895 µs


[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
